In [1]:
# mount Google Drive

from google.colab import drive
drive.mount('/content/drive')

Mounted at /content/drive


In [2]:
!pip install -q -U peft transformers datasets bitsandbytes trl==0.19.1 accelerate wandb

     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 41.7/41.7 kB 2.7 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 376.2/376.2 kB 25.6 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 11.2/11.2 MB 39.5 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 72.9/72.9 MB 9.5 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 363.4/363.4 MB 4.9 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 13.8/13.8 MB 52.4 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 24.6/24.6 MB 30.6 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 883.7/883.7 kB 38.7 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 664.8/664.8 MB 2.1 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 211.5/211.5 MB 7.4 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 56.3/56.3 MB 17.5 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 127.9/127.9 MB 7.8 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 

## Prep dataset

In [3]:
# prepare data
from datasets import load_dataset
eval_dataset = load_dataset("FreedomIntelligence/medical-o1-verifiable-problem", split="train[:1%]")

/usr/local/lib/python3.11/dist-packages/huggingface_hub/utils/_auth.py:104: UserWarning: 
Error while fetching `HF_TOKEN` secret value from your vault: 'Requesting secret HF_TOKEN timed out. Secrets can only be fetched when running from the Colab UI.'.
You are not authenticated with the Hugging Face Hub in this notebook.
If the error persists, please let us know by opening an issue on GitHub (https://github.com/huggingface/huggingface_hub/issues/new).
  warnings.warn(


README.md: 0.00B [00:00, ?B/s]

medical_o1_verifiable_problem.json:   0%|          | 0.00/12.2M [00:00<?, ?B/s]

Generating train split:   0%|          | 0/40644 [00:00<?, ? examples/s]

In [4]:
eval_dataset

Dataset({
    features: ['Open-ended Verifiable Question', 'Ground-True Answer'],
    num_rows: 406
})

In [5]:
import pandas as pd
eval_dataset = pd.DataFrame(eval_dataset)
eval_dataset.head()

,Open-ended Verifiable Question,Ground-True Answer
0,An 88-year-old woman with osteoarthritis is ex...,Gastric ulcer
1,In the context of disseminated intravascular c...,Fibrin degradation products
2,"In a 3-year-old boy with severe diarrhea, vomi...","Double-stranded, icosahedral, non-enveloped"
3,Based on the chest radiograph and abdominal CT...,Hydatid Cyst
4,What is one potential side effect that is not ...,Anaphylaxis


## Set up model definition

In [6]:
# load model
import torch
from transformers import AutoTokenizer, AutoModelForCausalLM, BitsAndBytesConfig
from google.colab import userdata

# Load the LoRA model
model_id = "erica-landreth/Meta-Llama-3-8B-Instruct-qlora-medical-alt-prompt"

quantization_config = BitsAndBytesConfig(
    load_in_4bit=True,
    bnb_4bit_compute_dtype=torch.float16,
    bnb_4bit_quant_type="nf4"
)

In [8]:
# Load tokenizer
base_model_id = "meta-llama/Meta-Llama-3-8B-Instruct"
tokenizer = AutoTokenizer.from_pretrained(base_model_id, trust_remote_code=True,
                                          token=userdata.get('HF_TOKEN'))
tokenizer.add_special_tokens({'pad_token': '<PAD>'})

tokenizer_config.json:   0%|          | 0.00/51.0k [00:00<?, ?B/s]

tokenizer.json:   0%|          | 0.00/9.09M [00:00<?, ?B/s]

special_tokens_map.json:   0%|          | 0.00/73.0 [00:00<?, ?B/s]

1

In [9]:
tokenizer.get_chat_template()

"{% set loop_messages = messages %}{% for message in loop_messages %}{% set content = '<|start_header_id|>' + message['role'] + '<|end_header_id|>\n\n'+ message['content'] | trim + '<|eot_id|>' %}{% if loop.index0 == 0 %}{% set content = bos_token + content %}{% endif %}{{ content }}{% endfor %}{% if add_generation_prompt %}{{ '<|start_header_id|>assistant<|end_header_id|>\n\n' }}{% endif %}"

## Eval of LoRA SFT (R = 8) model

In [10]:
# set up inference pipeline
from transformers import pipeline

pipe = pipeline("text-generation",
                model=model_id,
                model_kwargs={
                    "quantization_config":quantization_config,
                },
                token=userdata.get('notebook2'))


adapter_config.json:   0%|          | 0.00/916 [00:00<?, ?B/s]

config.json:   0%|          | 0.00/654 [00:00<?, ?B/s]

model.safetensors.index.json:   0%|          | 0.00/23.9k [00:00<?, ?B/s]

Fetching 4 files:   0%|          | 0/4 [00:00<?, ?it/s]

model-00002-of-00004.safetensors:   0%|          | 0.00/5.00G [00:00<?, ?B/s]

...

model-00004-of-00004.safetensors:   0%|          | 0.00/1.17G [00:00<?, ?B/s]

Loading checkpoint shards:   0%|          | 0/4 [00:00<?, ?it/s]

generation_config.json:   0%|          | 0.00/187 [00:00<?, ?B/s]

adapter_model.safetensors:   0%|          | 0.00/83.9M [00:00<?, ?B/s]

Device set to use cuda:0


In [11]:
fn_out = '/content/drive/MyDrive/w266_project/lora-sft-r8-output-reformatted-reference.txt'
with open(fn_out, 'w') as f:
    f.write('\n')

In [12]:
for idx,sample in enumerate(eval_dataset['Open-ended Verifiable Question']):

  prompt = [
      {"role":"system","content":"You are a medical expert"},
      {"role":"user","content": "Provide a succinct answer to the following question: "+sample}
  ]

  outputs = pipe(
      prompt,
      max_new_tokens=1024,
      temperature=0.01,
      return_full_text=False
  )
  response = outputs[0]["generated_text"]
  response = response.replace('\n','\t')
  print(response)
  with open(fn_out, 'a') as f:
      f.write(f"{idx}*****!!!!!"+response+'\n')

  if idx > 100:
    break

Setting `pad_token_id` to `eos_token_id`:128001 for open-end generation.
Setting `pad_token_id` to `eos_token_id`:128001 for open-end generation.


The most likely cause of the gastrointestinal blood loss in this 88-year-old woman is a gastric ulcer. Naproxen, a nonsteroidal anti-inflammatory drug (NSAID), is known to increase the risk of gastric ulcers due to its inhibitory effect on prostaglandins, which help protect the stomach lining. This risk is particularly significant in older adults, as they may have reduced stomach acid production, making them more susceptible to ulcers. The coffee-ground-like vomit is indicative of blood loss from the stomach, aligning with the potential for a gastric ulcer caused by naproxen use.


Setting `pad_token_id` to `eos_token_id`:128001 for open-end generation.


In the context of disseminated intravascular coagulation (DIC), the blood component expected to show an increase due to the excessive breakdown of fibrin is fibrinogen. This occurs because the body's attempt to compensate for the fibrin breakdown leads to an increase in fibrinogen production, which is then consumed by the coagulation cascade.


Setting `pad_token_id` to `eos_token_id`:128001 for open-end generation.


...

The primary mechanism of action for the analgesic drug likely prescribed to this patient is the inhibition of the reuptake of serotonin and norepinephrine. This mechanism is characteristic of drugs like tramadol, which are known for their effectiveness in managing chronic pain while also addressing opioid use disorders.


Setting `pad_token_id` to `eos_token_id`:128001 for open-end generation.


The appropriate surgical management for this patient with a solitary papillary thyroid carcinoma, as indicated by the 'orphan Annie-eyed nuclei' on FNAC, is a total thyroidectomy. This approach ensures complete removal of the thyroid gland, thereby preventing any potential recurrence.


Setting `pad_token_id` to `eos_token_id`:128001 for open-end generation.


Based on the clinical findings and laboratory results provided, a stone is most likely to be found in the common bile duct on ultrasonography. The presence of scleral icterus, elevated bilirubin, and elevated alkaline phosphatase and γ-glutamyltransferase levels suggest a blockage in the bile duct, which is consistent with a stone in the common bile duct.
Based on the symptoms described, the most likely diagnosis for the 23-year-old male's recurrent scaly lesions on the glans penis is balanitis xerotica obliterans (BXO). This condition is characterized by the appearance of lesions at the same location, which can be itchy and scaly, and often heal with hyperpigmentation. BXO is a common cause of these symptoms and is known for its distinctive features.
